In [21]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

URL = "https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm"

options = webdriver.ChromeOptions()
options.add_argument("--window-size=1280,900")
# options.add_argument("--headless=new")  # opcional

service = Service()
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 20)

try:
    driver.get(URL)

    # troca para iframe correto
    iframe = wait.until(EC.presence_of_element_located((By.ID, "bvmf_iframe")))
    driver.switch_to.frame(iframe)

    # seleciona "Novo Mercado"
    select_elem = wait.until(EC.presence_of_element_located((By.ID, "selectGovernance")))
    Select(select_elem).select_by_value("18")
    driver.execute_script("arguments[0].dispatchEvent(new Event('change'))", select_elem)

    # clica no botão Buscar
    buscar_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn-primary")))
    buscar_btn.click()

    # espera a tabela carregar
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.table")))
    time.sleep(2)

    # clica no botão lista para exibir todos os registros
    try:
        lista_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "i.fas.fa-list")))
        driver.execute_script("arguments[0].click();", lista_btn)
        time.sleep(2)
    except:
        print("⚠️ Botão de lista não encontrado ou não clicável. Continuando com tabela normal.")

    # função para extrair dados da tabela atual
    def extrai_tabela():
        dados = []
        rows = driver.find_elements(By.CSS_SELECTOR, "table.table tbody tr")
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) >= 4:
                razao = cols[0].text.strip()
                nome_pregao = cols[1].text.strip()
                segmento = cols[2].text.strip()
                codigo = cols[3].text.strip()
                dados.append([razao, nome_pregao, segmento, codigo])
        return dados

    # loop para percorrer todas as páginas
    all_data = []
    pagina_atual = 1
    while True:
        print(f"Extraindo página {pagina_atual}...")
        all_data.extend(extrai_tabela())

        # encontra o botão Next
        try:
            next_li = driver.find_element(By.CSS_SELECTOR, "li.pagination-next")
            classes = next_li.get_attribute("class")
            if "disabled" in classes:
                print("✅ Última página atingida.")
                break

            # clica no <a> dentro do <li>
            next_a = next_li.find_element(By.TAG_NAME, "a")
            driver.execute_script("arguments[0].click();", next_a)
            time.sleep(2)  # espera tabela atualizar
            pagina_atual += 1

        except Exception as e:
            print("⚠️ Não foi possível clicar na próxima página:", e)
            break

    # salva CSV
    df = pd.DataFrame(all_data, columns=["Razao_Social", "Nome_de_Pregao", "Segmento", "Codigo"])
    df.to_csv("empresas_novo_mercado.csv", index=False, encoding="utf-8-sig")
    print(f"✅ CSV salvo com {len(df)} empresas do Novo Mercado.")

finally:
    driver.quit()

Extraindo página 1...
Extraindo página 2...
Extraindo página 3...
Extraindo página 4...
Extraindo página 5...
Extraindo página 6...
Extraindo página 7...
Extraindo página 8...
Extraindo página 9...
Extraindo página 10...
✅ Última página atingida.
✅ CSV salvo com 186 empresas do Novo Mercado.


In [22]:
df

,Razao_Social,Nome_de_Pregao,Segmento,Codigo
0,AERIS IND. E COM. DE EQUIP. GERACAO DE ENERGIA...,AERIS,NM,AERI
1,AGROGALAXY PARTICIPAÇÕES S.A.,AGROGALAXY,NM,AGXY
2,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,ALLIAR,NM,AALR
3,ALLIED TECNOLOGIA S.A.,ALLIED,NM,ALLD
4,ALLOS S.A,ALLOS,NM,ALOS
...,...,...,...,...
181,WEG S.A.,WEG,NM,WEGE
182,WESTWING COMERCIO VAREJISTA S.A.,WESTWING,NM,WEST
183,WILSON SONS S.A.,WILSON SONS,NM,PORT
184,WIZ CO PARTICIPAÇÕES E CORRETAGEM DE SEGUROS S.A.,WIZ CO,NM,WIZC
